$$ \LARGE\bf{EXAM \ PROJECT \ ISDS 2022} $$

$\textbf{Imports}$

In [1]:
import requests
import json
import numbers
import pandas as pd
from bs4 import BeautifulSoup
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

$\textbf{Data}$, housing prices boliga.dk - ONLY FOR FIRST PAGE

In [2]:
#1. Load page boliga.dk
url_boliga = "https://www.boliga.dk/resultat?propertyType=3"
response_boliga = requests.get(url_boliga,headers={'name':'Sevda Öztürk','email':'txg194b@alumni.ku.dk'})
soup = BeautifulSoup(response_boliga.content,"lxml")

#2. Scrape housing prices from www.boliga.dk
list_price = soup.find_all("div" ,class_ = "price")
price = [col.text.split(u'\xa0')[0] for col in list_price]

$\bf{Data}$, address, zipcode from boliga.dk -  ONLY FOR FIRST PAGE

In [3]:
#1. Scrape address from www.boliga.dk (first page)
list_address = soup.find_all("div" ,class_ = 'w-100 mt-1 listing-address font-weight-bold text-nowrap text-truncate')
address = [col.text.split(',',1)[0] for col in list_address]

#2. Scrape zipcode from www.boliga.dk (first page)
list_zipcode = soup.find_all("div" ,class_ = "zip-color text-nowrap text-truncate")
zipcode = [col.text for col in list_zipcode]

$\textbf{Data}$, m2, rooms from boliga.dk -  ONLY FOR FIRST PAGE

In [4]:
#1. Scrape size of houses
list_m2 = soup.find_all("span" ,class_ = "text-nowrap")
m2 = [col.text.split() for col in list_m2]

#the variable m2 is a list of lists. The following code keeps the first element of every sublist to get rid of the string "m2"
m2 = [element[0] for element in m2]

del m2[0] #delete first element in m2 in order to get all m2 in next code
m2 = m2[::6] #we now keep every 6th element, which is the size of every address

#2. Scrape number of rooms and prepare
list_rooms = soup.find_all("span" ,class_ = "text-nowrap")
rooms = [col.text.split() for col in list_rooms]
rooms = rooms[::6]
rooms = [element[1] for element in rooms]
rooms

for i in range(len(rooms)):
    if rooms[i] == 'værelse':
        rooms[i] = '1'

$\bf{Combine \ data \ in \ one \ dataset}$

In [5]:
#1. Combine all scraped lists in one dataframe
df_addzip = {"Price": price, "Address": address, "Zipcode": zipcode, "m2": m2, "rooms" : rooms}
df_data = pd.DataFrame(df_addzip)
#df_data

$\bf{Use \ combined \ dataset \ to \ get \ x,y \ coordinates}$, distance to nearest (daycare eksempel) 

In [6]:
#1. Get longitude and latitude of each address in df_data["Address"]
z = []

for a in df_data["Address"]:    
    response_dist = requests.get(f"https://api.dataforsyningen.dk/adresser?q={a}&struktur=mini",headers={'name':'Sevda Öztürk','email':'txg194@alumni.ku.dk'})
    res_j = response_dist.json()
    z.append(res_j)

#2 Get longitude, x, and latitudes, y from address i
length = list(range(len(z)))
x = []
y = []

for i in length:
    xx = z[i][0]["x"]
    x.append(xx)
    
for i in length:
    yy = z[i][0]["y"]
    y.append(yy)

In [7]:
#3. Add x and y to dataset df_data
df_data["x"], df_data["y"] = x, y
#df_data["yx"] = df_data['y'] + ',' + df_data['x']
df_data

,Price,Address,Zipcode,m2,rooms,x,y
0,595.000,Grøfthøjparken 151,8260 Viby J,33,1,10.136680,56.123838
1,1.248.000,Ingemanns Alle 175B,6700 Esbjerg,72,2,8.473389,55.472914
2,1.345.000,Højagerparken 19,2750 Ballerup,46,1,10.039818,56.212746
3,1.399.000,Lotusvej 18,5250 Odense SV,91,4,12.146738,55.791236
4,1.895.000,Sthensvej 7A,2630 Taastrup,54,1,12.299210,55.655865
5,1.900.000,Hjelmensgade 22,8000 Aarhus C,66,2,10.209005,56.162319
6,2.150.000,Roskildevænget 6,4000 Roskilde,71,3,12.092344,55.627380
7,2.195.000,Borgergade 23A,8700 Horsens,134,5,9.855443,55.861479
8,2.295.000,Buegården 5,2880 Bagsværd,60,2,12.457757,55.760414
9,2.395.000,Skjoldagervej 72,2820 Gentofte,57,2,12.528949,55.760907


$\bf{Use \ coordinates \ to \ get \ distance}$

In [9]:
dist = []
for x,y in df_data:
    afstand_daycare_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={df_data["y"]},{df_data["x"]}&poitypes=daycare&decodepolyline')
    afstand_daycare_json.json()


ValueError: too many values to unpack (expected 2)